# Thruster experiment Overview

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import statsmodels.api as sm

from master_thesis.visualization import visualize
import scipy.integrate

from master_thesis.pipelines.trip_statistics import clean_statistics
from scipy.stats import norm
#from myst_nb import glue
#plt.style.use('presentation')
from IPython.display import display, Markdown, Latex
import sympy as sp

In [ ]:
%reload_kedro
#df_stat = catalog.load('uraniborg.trip_statistics_clean')
df_stat = catalog.load('uraniborg.steaming.trip_statistics')
df_stat['start_time'] = pd.to_datetime(df_stat['start_time'], utc=True)
df_stat['end_time'] = pd.to_datetime(df_stat['end_time'], utc=True);

df_stat['start_time'] = df_stat['start_time'].apply(lambda x : x.tz_convert(tz='Europe/Berlin'))
df_stat['end_time'] = df_stat['end_time'].apply(lambda x : x.tz_convert(tz='Europe/Berlin'))
#df_stat.describe()

Statistics for steaming data (excluding in port manoeuvres) between the points in the figure:

In [ ]:
data_steaming = catalog.load("uraniborg.data_steaming")
trip_no = list(data_steaming.groupby(by='trip_no').groups.keys())[0]
df = data_steaming.groupby(by='trip_no', sort=False).get_group(trip_no)
visualize.plot_map(df)

In [ ]:
df_clean = df_stat.copy()

Time history of fuel consumption for all trips during the experiment.

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,5)
facegrid = sns.scatterplot(data=df_clean, x='start_time',y='Consumption ME (L)', ax=ax);
ax.set_ylabel('(Fuel consumption per trip) [l]');

fig = ax.get_figure()
fig_name = 'overview'

In [ ]:
start_experiment = pd.to_datetime("2022-08-19 14:15:00+2", utc=False).tz_convert(tz='Europe/Berlin')
end_experiment = pd.to_datetime("2022-08-22 06:50:00+2", utc=False).tz_convert(tz='Europe/Berlin')

start_index = (df_clean['start_time'] - start_experiment).abs().idxmin()
end_index = (df_clean['start_time'] - end_experiment).abs().idxmin()

df_clean=df_clean.loc[start_index:end_index].copy()
steps = df_clean['trip_no'] - df_clean.loc[start_index]['trip_no']

In [ ]:
super_trips = np.floor(steps / 2)

In [ ]:
df_clean['operation'] = super_trips.apply(lambda x : 'ÖST' if x % 2 == 0 else 'BFH')

In [ ]:
%reload_kedro
time_table = catalog.load("uraniborg.time_table")
time_table.index = time_table.index.tz_convert(tz='Europe/Berlin')
assert (df_clean['operation'] == time_table['operation'].values).all()
assert (df_clean['trip_direction'] == time_table['direction'].values).all()

Uraniborg is run with the following time table during the experiment. The operation of the ship is switching every other roundtrip Landskrona-Ven-Landskrona between the captain (BFH) and by the chief mate (ÖST).

In [ ]:
time_table

In [ ]:
df_clean['time_table'] = time_table.index.tz_convert(tz='Europe/Berlin')

In [ ]:
df_clean['delay'] = (df_clean['start_time']) - df_clean['time_table']

In [ ]:
assert (df_clean['delay'] < pd.to_timedelta("10T")).all()

The trips operated by ÖST and BFH are shown below.

In [ ]:
px.scatter(data_frame=df_clean, x='start_time', y='Consumption ME (L)', color='operation')

The consumption is plotted vs. the power ratio (PR) in the figure below. PR=1 means that only the aft thruster is run.

In [ ]:
px.scatter(data_frame=df_clean, x='PR', y='Consumption ME (L)', color='operation', hover_data=["time_table","sog"])

There is however a difference in the average speed over ground (sog) between ÖST and BFH as seen in the figure below.

In [ ]:
px.scatter(data_frame=df_clean, x='PR', y='sog', color='operation', hover_data=["time_table"])

To make a more fair comparison, some trips are excluded so that the mean of the average trip speed for ÖST and BFH is the same:

In [ ]:
mask = ((df_clean['operation'] == 'BFH') & (df_clean['PR'] > 0.9) & (df_clean['sog'] > 4.5))
df_experiment_sog = df_clean.loc[mask].copy()
df_clean['speed classify'] = np.NaN
df_clean.loc[mask,'speed classify'] = 'experiment'

df_experiment_sog.mean(numeric_only=True)[['sog','Consumption ME (L)']]


In [ ]:
mask = ((df_clean['operation'] == 'ÖST') & (df_clean['PR'] < 0.9) & (df_clean['sog'] < 4.95))
df_normal_sog = df_clean.loc[mask].copy()
df_clean.loc[mask,'speed classify'] = 'normal'
df_clean['speed classify'] = df_clean['speed classify'].fillna('excluded')
df_normal_sog.mean(numeric_only=True)[['sog','Consumption ME (L)']]

In [ ]:
(df_normal_sog['Consumption ME (L)'].mean() - df_experiment_sog['Consumption ME (L)'].mean())/df_normal_sog['Consumption ME (L)'].mean()


In [ ]:
px.scatter(data_frame=df_clean, x='PR', y='Consumption ME (L)', color='speed classify', hover_data=["time_table","sog"])

## Conclusion

BFH, which is utlizing more of the aft thruster, has **14%** lower fuel consumption than ÖST. If this improvement originate entirely from the changed thrust allocation is not certain. Perhaps BFH is also a more skilled operator than ÖST from the start? Trips prior to this experiment made by BFH and ÖST will be further investigated to also see if this is the case. Regardless of this the present result show that it should be possible to reduce the fuel consumption by around 20%, also accounting for other improvments when comparing the operation of BFH and ÖST, such as speed and speed profile etc.